In [1]:
import pandas as pd
import numpy as np
import typing as tp

from datetime import datetime, timedelta
from dateutil.parser import parse
from scipy.stats import levy_stable

In [2]:
# Import csv data for last <= 120 days of priceCumulative values from csv/data-1626472963.csv
DATA_FILENAME = "data-1626472963"
df_raw = pd.read_csv(f"csv/{DATA_FILENAME}.csv")
df_raw

,result,table,_start,_stop,_time,_value,_field,_measurement,id,influx-sushi,type,token0_name,token1_name
0,_result,0,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-04-18 18:52:55+00:00,2.899194e+52,price0Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,metrics-hourly,NaN,NaN
1,_result,0,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-04-18 18:56:00+00:00,2.899218e+52,price0Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,metrics-hourly,NaN,NaN
2,_result,0,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-04-18 19:46:47+00:00,2.899617e+52,price0Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,metrics-hourly,NaN,NaN
3,_result,0,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-04-18 20:30:01+00:00,2.899956e+52,price0Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,metrics-hourly,NaN,NaN
4,_result,0,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-04-18 20:46:59+00:00,2.900089e+52,price0Cumulative,mem,Sushiswap: WETH / WBTC,ingest-data-frame,metrics-hourly,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241229,_result,57,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-07-16 21:07:28+00:00,1.619394e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC
241230,_result,57,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-07-16 21:24:01+00:00,1.619422e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC
241231,_result,57,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-07-16 21:35:31+00:00,1.619440e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC
241232,_result,57,2021-03-18 22:02:38.914870+00:00,2021-07-16 22:02:38.914870+00:00,2021-07-16 21:46:19+00:00,1.619458e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC


In [3]:
# Filter for cols we care about: [id, _time, _field, _value] and sort by time
df = df_raw.filter(items=['id', '_time', '_field', '_value'])
df = df.sort_values(by='_time', ignore_index=True)
df

,id,_time,_field,_value
0,Sushiswap: COMP / WETH,2021-04-18 17:47:35+00:00,price1Cumulative,4.148446e+41
1,Sushiswap: COMP / WETH,2021-04-18 17:47:35+00:00,price0Cumulative,2.528080e+40
2,Sushiswap: UNI / WETH,2021-04-18 18:31:43+00:00,price0Cumulative,9.936822e+38
3,Sushiswap: UNI / WETH,2021-04-18 18:31:43+00:00,price1Cumulative,1.109690e+43
4,Sushiswap: ALPHA / WETH,2021-04-18 18:42:40+00:00,price1Cumulative,8.288763e+43
...,...,...,...,...
241229,Sushiswap: CRV / WETH,2021-07-16 22:00:50+00:00,price0Cumulative,1.245158e+44
241230,Sushiswap: WETH / DAI,2021-07-16 22:01:02+00:00,price0Cumulative,1.630421e+38
241231,Sushiswap: WETH / DAI,2021-07-16 22:01:02+00:00,price1Cumulative,2.061793e+44
241232,Sushiswap: WETH / USDC,2021-07-16 22:01:02+00:00,price1Cumulative,2.065081e+32


In [4]:
# Examine stats for pairs: WETH / DAI (less volatile) ...
df_weth_dai = df[(df['id'] == "Sushiswap: WETH / DAI") & (df['_field'] == "price1Cumulative")]
df_weth_dai

,id,_time,_field,_value
81364,Sushiswap: WETH / DAI,2021-05-19 20:47:17+00:00,price1Cumulative,1.459467e+44
81399,Sushiswap: WETH / DAI,2021-05-19 21:07:02+00:00,price1Cumulative,1.459625e+44
81419,Sushiswap: WETH / DAI,2021-05-19 21:17:59+00:00,price1Cumulative,1.459719e+44
81448,Sushiswap: WETH / DAI,2021-05-19 21:28:19+00:00,price1Cumulative,1.459791e+44
81488,Sushiswap: WETH / DAI,2021-05-19 21:40:59+00:00,price1Cumulative,1.459890e+44
...,...,...,...,...
241140,Sushiswap: WETH / DAI,2021-07-16 21:12:04+00:00,price1Cumulative,2.061504e+44
241157,Sushiswap: WETH / DAI,2021-07-16 21:23:53+00:00,price1Cumulative,2.061574e+44
241178,Sushiswap: WETH / DAI,2021-07-16 21:33:53+00:00,price1Cumulative,2.061633e+44
241204,Sushiswap: WETH / DAI,2021-07-16 21:45:39+00:00,price1Cumulative,2.061702e+44


In [5]:
# ... and WETH / USDC (less volatile, but we have more data than above) ...
df_weth_usdc = df[(df['id'] == "Sushiswap: WETH / USDC") & (df['_field'] == "price1Cumulative")]
df_weth_usdc

,id,_time,_field,_value
21,Sushiswap: WETH / USDC,2021-04-18 18:52:52+00:00,price1Cumulative,1.029267e+32
40,Sushiswap: WETH / USDC,2021-04-18 18:56:00+00:00,price1Cumulative,1.029289e+32
58,Sushiswap: WETH / USDC,2021-04-18 19:55:50+00:00,price1Cumulative,1.029708e+32
90,Sushiswap: WETH / USDC,2021-04-18 20:30:01+00:00,price1Cumulative,1.029945e+32
111,Sushiswap: WETH / USDC,2021-04-18 20:40:49+00:00,price1Cumulative,1.030020e+32
...,...,...,...,...
241133,Sushiswap: WETH / USDC,2021-07-16 21:07:28+00:00,price1Cumulative,2.064765e+32
241158,Sushiswap: WETH / USDC,2021-07-16 21:24:01+00:00,price1Cumulative,2.064862e+32
241182,Sushiswap: WETH / USDC,2021-07-16 21:35:31+00:00,price1Cumulative,2.064930e+32
241212,Sushiswap: WETH / USDC,2021-07-16 21:46:19+00:00,price1Cumulative,2.064994e+32


In [6]:
# ... and ALCX / WETH (more volatile)
df_alcx_weth = df[(df['id'] == "Sushiswap: ALCX / WETH") & (df['_field'] == "price1Cumulative")]
df_alcx_weth

,id,_time,_field,_value
8,Sushiswap: ALCX / WETH,2021-04-18 18:50:49+00:00,price1Cumulative,1.831283e+40
33,Sushiswap: ALCX / WETH,2021-04-18 18:55:04+00:00,price1Cumulative,1.831382e+40
56,Sushiswap: ALCX / WETH,2021-04-18 19:55:02+00:00,price1Cumulative,1.832779e+40
87,Sushiswap: ALCX / WETH,2021-04-18 20:28:40+00:00,price1Cumulative,1.833563e+40
102,Sushiswap: ALCX / WETH,2021-04-18 20:38:26+00:00,price1Cumulative,1.833792e+40
...,...,...,...,...
241108,Sushiswap: ALCX / WETH,2021-07-16 20:54:15+00:00,price1Cumulative,3.101423e+40
241150,Sushiswap: ALCX / WETH,2021-07-16 21:21:17+00:00,price1Cumulative,3.101542e+40
241173,Sushiswap: ALCX / WETH,2021-07-16 21:32:52+00:00,price1Cumulative,3.101593e+40
241193,Sushiswap: ALCX / WETH,2021-07-16 21:42:58+00:00,price1Cumulative,3.101637e+40


In [7]:
# ... and UNI / WETH (mid volatile)
df_uni_weth = df[(df['id'] == "Sushiswap: UNI / WETH") & (df['_field'] == "price0Cumulative")]
df_uni_weth

,id,_time,_field,_value
2,Sushiswap: UNI / WETH,2021-04-18 18:31:43+00:00,price0Cumulative,9.936822e+38
35,Sushiswap: UNI / WETH,2021-04-18 18:55:04+00:00,price0Cumulative,9.937873e+38
67,Sushiswap: UNI / WETH,2021-04-18 19:58:10+00:00,price0Cumulative,9.940690e+38
72,Sushiswap: UNI / WETH,2021-04-18 20:13:22+00:00,price0Cumulative,9.941360e+38
131,Sushiswap: UNI / WETH,2021-04-18 20:51:09+00:00,price0Cumulative,9.943018e+38
...,...,...,...,...
240993,Sushiswap: UNI / WETH,2021-07-16 19:51:04+00:00,price0Cumulative,1.408277e+39
241029,Sushiswap: UNI / WETH,2021-07-16 20:12:20+00:00,price0Cumulative,1.408335e+39
241171,Sushiswap: UNI / WETH,2021-07-16 21:32:39+00:00,price0Cumulative,1.408555e+39
241188,Sushiswap: UNI / WETH,2021-07-16 21:37:51+00:00,price0Cumulative,1.408569e+39


In [8]:
# TODO:
#  1. Compute the TWAP from PC value in df [x]
#  2. Fit Levy stable with scipy built in MLE code [x]
#  3. Report Levy stable params fit (a, b, mu, sig) [x]
#  4. Calc [e**(mu*t + sig * (t/a)**(1/a) * F^{-1}(1-alpha)) - 1] (VaR * d**m normalized for imbalance) for diff t vals
#  5. See how large/small C_p = 4. can be for various t, alpha combos

In [8]:
# Define some functions to calc twap
PC_RESOLUTION = 112

def compute_amount_out(twap_112: np.ndarray, amount_in: int) -> np.ndarray:
    rshift = np.vectorize(lambda x: int(x * amount_in) >> PC_RESOLUTION)
    return rshift(twap_112)

def get_twap(pc: pd.DataFrame, window: int, amount_in: int) -> pd.DataFrame:
    dp = pc.filter(items=['_value'])\
        .rolling(window=window)\
        .apply(lambda w: w[-1] - w[0], raw=True)
    
    # for time, need to map to timestamp first then apply delta
    dt = pc.filter(items=['_time'])\
        .applymap(parse)\
        .applymap(datetime.timestamp)\
        .rolling(window=window)\
        .apply(lambda w: w[-1] - w[0], raw=True)

    # with NaNs filtered out
    twap_112 = (dp['_value'] / dt['_time']).to_numpy()
    twap_112 = twap_112[np.logical_not(np.isnan(twap_112))]
    twaps = compute_amount_out(twap_112, amount_in)
    
    # window close timestamps
    t = pc.filter(items=['_time'])\
        .applymap(parse)\
        .applymap(datetime.timestamp)\
        .rolling(window=window)\
        .apply(lambda w: w[-1], raw=True)
    ts = t['_time'].to_numpy()
    ts = ts[np.logical_not(np.isnan(ts))]
    
    df = pd.DataFrame(data=[ts, twaps]).T
    df.columns = ['timestamp', 'twap']
    
    # filter out any twaps that are less than or equal to 0
    df = df[df['twap'] > 0]
    return df

In [9]:
# 1. Compute the TWAP from PC value in df
WINDOW = 6
UNIT_WETH = 1e18

df_weth_dai_twap = get_twap(pc=df_weth_dai, window=WINDOW, amount_in=UNIT_WETH)
df_weth_usdc_twap = get_twap(pc=df_weth_usdc, window=WINDOW, amount_in=UNIT_WETH)
df_alcx_weth_twap = get_twap(pc=df_alcx_weth, window=WINDOW, amount_in=UNIT_WETH)
df_uni_weth_twap = get_twap(pc=df_uni_weth, window=WINDOW, amount_in=UNIT_WETH)

In [10]:
df_weth_dai_twap

,timestamp,twap
0,1.62146e+09,2530192649420577177600
1,1.62146e+09,2526661690450846416896
2,1.62146e+09,2504822078413549862912
3,1.62146e+09,2580894455288732057600
4,1.62146e+09,2610532884887874043904
...,...,...
7298,1.62647e+09,1906330357480768143360
7299,1.62647e+09,1904218942956549963776
7300,1.62647e+09,1902239014663740719104
7301,1.62647e+09,1899809813543781400576


In [11]:
df_weth_dai_twap['twap'] / UNIT_WETH

0       2530.19
1       2526.66
2       2504.82
3       2580.89
4       2610.53
         ...   
7298    1906.33
7299    1904.22
7300    1902.24
7301    1899.81
7302    1897.95
Name: twap, Length: 7303, dtype: object

In [12]:
df_weth_usdc_twap

,timestamp,twap
0,1.618779e+09,2.239566e+09
1,1.618780e+09,2.240544e+09
2,1.618780e+09,2.238170e+09
3,1.618781e+09,2.250825e+09
4,1.618782e+09,2.256557e+09
...,...,...
11267,1.626470e+09,1.909707e+09
11268,1.626471e+09,1.905958e+09
11269,1.626471e+09,1.902865e+09
11270,1.626472e+09,1.900337e+09


In [13]:
# Export twaps to csv to analyze in mathematica as well
df_weth_dai_twap.to_csv(f"csv/{DATA_FILENAME}_weth-dai-twap.csv")
df_weth_usdc_twap.to_csv(f"csv/{DATA_FILENAME}_weth-usdc-twap.csv")
df_alcx_weth_twap.to_csv(f"csv/{DATA_FILENAME}_alcx-weth-twap.csv")
df_uni_weth_twap.to_csv(f"csv/{DATA_FILENAME}_uni-weth-twap.csv")

In [12]:
# Define some functions to fit TWAP to log stable
def get_rs(twap: pd.DataFrame) -> np.ndarray:
    sample = twap['twap'].to_numpy()
    return [
        np.log(sample[i]/sample[i-1])
        for i in range(1, len(sample), 1)
    ]

def fit_to_log_stable(twap: pd.DataFrame) -> tp.Tuple[float]:
    rs = get_rs(twap)
    return levy_stable.fit(rs)

def get_params(twap: pd.DataFrame, period: int) -> pd.DataFrame:
    a, b, loc, scale = fit_to_log_stable(twap)

    # transform to mu, sig
    mu = loc / period
    sig = scale / (period/a)**(1/a)
    
    df = pd.DataFrame(data=[a, b, mu, sig]).T
    df.columns = ['a', 'b', 'mu', 'sig']
    return df

In [13]:
# 2. Fit Levy stable with scipy built in MLE code
PERIOD = 600 # 10 min

# NOTE: this is taking too long ... what's a way around this while still using scipy?
# Could wrap C++ code from Nolan instead ...
# TODO: export df_twap to csv and use mathematica to check fitting is ok (is scipy issue)
# df_weth_dai_params = get_params(twap=df_weth_dai_twap, period=PERIOD)
# df_alcx_weth_params = get_params(twap=df_alcx_weth_twap, period=PERIOD)
# df_uni_weth_params = get_params(twap=df_uni_weth_twap, period=PERIOD)

In [14]:
# 3. Report Levy stable params fit (a, b, mu, sig)
# df_weth_dai_params

NameError: name 'df_weth_dai_params' is not defined

In [4]:
#stables = np.array([
#    (a, b, mu, c)
#    for a in np.arange(0.75, 2.01, 0.25)
#    for b in np.arange(-0.5, 1.0, 0.5)
#    for mu in np.arange(-0.1, 0.1, 0.01)
#    for c in np.arange(0.25, 1.01, 0.25)
#])
stables = np.array([1.3278285879842862,0.0816835526225623,-0.0000252748167384907,0.0006409442772706084])
#stables_df = pd.DataFrame(stables)
#stables_df.columns = ['alpha', 'beta', 'loc', 'scale']
#stables_df
stables

array([ 1.32782859e+00,  8.16835526e-02, -2.52748167e-05,  6.40944277e-04])

In [34]:
qs = np.arange(0.1, 1.0, 0.01)
qs
quantiles = [levy_stable.ppf(q, alpha=stables[0], beta=stables[1], loc=stables[2], scale=stables[3]) for q in qs]
quantiles
q_df = pd.DataFrame(data=[qs, quantiles]).T
q_df.columns = ['q', 'value']
q_df
q_df.to_csv('cdfs.csv')

In [23]:
cvs = np.arange(-0.01, 0.01, 0.0001)
cdfs = [levy_stable.cdf(cv, alpha=stables[0], beta=stables[1], loc=stables[2], scale=stables[3]) for cv in cvs]
cdf_df = pd.DataFrame(data=[cvs, cdfs]).T
cdf_df.columns = ['x', 'value']
cdf_df
cdf_df.to_csv('cdfs.csv')

In [27]:
pvs = np.arange(-0.01, 0.01, 0.0001)
pdfs = [levy_stable.pdf(pv, alpha=stables[0], beta=stables[1], loc=stables[2], scale=stables[3]) for pv in pvs]
pdf_df = pd.DataFrame(data=[pvs, pdfs]).T
pdf_df.columns = ['x', 'value']
pdf_df
pdf_df.to_csv('pdfs.csv')

In [31]:
pdf_df.to_numpy().tolist()

[[-0.01, 0.8342282418569105],
 [-0.0099, 0.8546360956283016],
 [-0.009800000000000001, 0.8757669443572541],
 [-0.009700000000000002, 0.897654515108729],
 [-0.009600000000000003, 0.9203344952362921],
 [-0.009500000000000003, 0.9438446694234918],
 [-0.009400000000000004, 0.9682250677971895],
 [-0.009300000000000004, 0.993518126326782],
 [-0.009200000000000005, 1.019768860648664],
 [-0.009100000000000006, 1.0470250546284132],
 [-0.009000000000000006, 1.0753374649684941],
 [-0.008900000000000007, 1.1047600437659002],
 [-0.008800000000000007, 1.135350180367068],
 [-0.008700000000000008, 1.1671689648052055],
 [-0.008600000000000009, 1.2002814748850732],
 [-0.00850000000000001, 1.234757089383081],
 [-0.00840000000000001, 1.2706698300957142],
 [-0.00830000000000001, 1.3080987357935758],
 [-0.008200000000000011, 1.3471282715055983],
 [-0.008100000000000012, 1.3878487769747183],
 [-0.008000000000000012, 1.430356958599867],
 [-0.007900000000000013, 1.474756429718102],
 [-0.0078000000000000135, 1.